In [1]:
import json
import random
import re

# 멘토와 멘티 이름 리스트
mentor_names = [
    "김민수", "이서연", "박지훈", "정예은", "최현우", "강수진", "한동훈", "오세영", "최지훈", "박소연",
    "윤동주", "김수현", "이영희", "박찬호", "조민준", "서지우", "황준호", "장예진", "류지성", "신나리",
    "오민아", "최서준", "백지민", "송다은", "문지훈", "전민수", "정하영", "한예슬", "홍길동", "강민주",
    "김유진", "이준호", "박서영", "최민호", "정다은", "조윤서", "강민우", "신지훈", "오하늘", "한서진",
    "서예지", "윤민호", "이수빈", "박지민", "김다은", "정현우", "최수진", "조민호", "강지은", "오준서"
]

mentee_names = [
    "김하늘", "오준호", "신지아", "한지민", "서현진", "이준서", "박채원", "조민수", "윤소희", "장민준",
    "이지은", "김도현", "최수아", "강예린", "남도현", "백현우", "송지은", "문지호", "전지현", "정수빈",
    "한예지", "홍서영", "류민지", "박준서", "강하늘", "윤채영", "이승현", "조아라", "오지훈", "신동엽",
    "김수빈", "박민주", "이서현", "최다인", "정재훈", "한지우", "조은서", "강준호", "윤하늘", "오지민",
    "서지훈", "김나연", "박성민", "이하은", "최유진", "정세훈", "한도현", "조민지", "강서연", "오유나"
]

# 레벨 목록
levels = {
    1: "초급",
    2: "중급",
    3: "고급"
}

# 카테고리와 중분류 정의
categories = {
    "학업/교육": {
        "진로상담": {
            "expertise": ["진로 설계", "직업 선택", "진로 전환"],
            "interests": ["진로 고민", "직업 탐색", "미래 계획"],
            "mentor_bios": [
                "{level} 수준의 진로 상담 전문가입니다. 여러분의 미래를 함께 고민하겠습니다.",
                "진로 설계 분야에서 {level} 멘토로 활동하고 있습니다. 방향을 찾아드리겠습니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 올바른 길을 찾고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 해결하고 싶습니다."
            ]
        },
        "취업 준비": {
            "expertise": ["이력서 작성", "자소서 작성", "취업 전략"],
            "interests": ["취업 준비", "이력서 개선", "취업 정보"],
            "mentor_bios": [
                "{level} 수준의 취업 준비 전문가입니다. 성공적인 취업을 도와드리겠습니다.",
                "취업 전략 분야에서 {level} 멘토로 활동하고 있습니다. 노하우를 전수해드리겠습니다."
            ],
            "mentee_bios": [
                "{interest}에 집중하고 있는 {level} 멘티입니다. 좋은 직장을 구하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 준비하고 있습니다."
            ]
        },
        "면접 대비": {
            "expertise": ["면접 기술", "프레젠테이션 스킬", "커뮤니케이션"],
            "interests": ["면접 준비", "스피치 능력", "자신감 향상"],
            "mentor_bios": [
                "{level} 수준의 면접 대비 전문가입니다. 면접에서 빛날 수 있도록 도와드리겠습니다.",
                "커뮤니케이션 분야에서 {level} 멘토로 활동하고 있습니다. 자신감을 심어드리겠습니다."
            ],
            "mentee_bios": [
                "{interest}에 도전하는 {level} 멘티입니다. 면접을 성공적으로 치르고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 향상시키고 싶습니다."
            ]
        },
        "대학입시 및 유학": {
            "expertise": ["대학 입시 전략", "유학 컨설팅", "시험 준비"],
            "interests": ["입시 정보", "유학 계획", "시험 대비"],
            "mentor_bios": [
                "{level} 수준의 입시 및 유학 전문가입니다. 꿈의 대학에 합격하도록 돕겠습니다.",
                "유학 컨설팅 분야에서 {level} 멘토로 활동하고 있습니다. 해외 유학을 준비해요."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 목표를 달성하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 준비하고 있습니다."
            ]
        },
        "자격증": {
            "expertise": ["자격증 취득 방법", "시험 대비 전략", "학습 계획"],
            "interests": ["자격증 공부", "시험 준비", "자기계발"],
            "mentor_bios": [
                "{level} 수준의 자격증 취득 전문가입니다. 합격의 비결을 알려드리겠습니다.",
                "시험 대비 분야에서 {level} 멘토로 활동하고 있습니다. 성공적인 학습을 지원합니다."
            ],
            "mentee_bios": [
                "{interest}에 열정을 가진 {level} 멘티입니다. 자격증을 취득하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        }
    },
    "경력/직업": {
        "경력전환": {
            "expertise": ["커리어 코칭", "직무 변경", "재교육"],
            "interests": ["새로운 분야 탐색", "경력 전환", "직무 교육"],
            "mentor_bios": [
                "{level} 수준의 경력 전환 전문가입니다. 새로운 길을 안내해드리겠습니다.",
                "커리어 코칭 분야에서 {level} 멘토로 활동하고 있습니다. 함께 성장합시다."
            ],
            "mentee_bios": [
                "{interest}에 도전하는 {level} 멘티입니다. 변화를 원해요.",
                "안녕하세요, {level} 단계의 {interest}를 고민하고 있습니다."
            ]
        },
        "직장인 실무": {
            "expertise": ["업무 효율화", "프로젝트 관리", "비즈니스 커뮤니케이션"],
            "interests": ["실무 능력 향상", "효율적인 일 처리", "팀워크"],
            "mentor_bios": [
                "{level} 수준의 직장인 실무 전문가입니다. 업무 능력을 향상시켜드리겠습니다.",
                "프로젝트 관리 분야에서 {level} 멘토로 활동하고 있습니다. 경험을 나누겠습니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 직장에서 성장하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        },
        "직장 내 문제 해결": {
            "expertise": ["갈등 관리", "스트레스 관리", "조직 문화 이해"],
            "interests": ["직장 문제 해결", "스트레스 해소", "긍정적인 관계"],
            "mentor_bios": [
                "{level} 수준의 직장 문제 해결 전문가입니다. 어려움을 함께 극복합시다.",
                "조직 문화 분야에서 {level} 멘토로 활동하고 있습니다. 조언을 드리겠습니다."
            ],
            "mentee_bios": [
                "{interest}를 원하고 있는 {level} 멘티입니다. 직장 생활을 개선하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 찾고 있습니다."
            ]
        },
        "리더십 및 팀 관리": {
            "expertise": ["리더십 개발", "팀 빌딩", "성과 관리"],
            "interests": ["리더십 배양", "팀 관리 기술", "성과 향상"],
            "mentor_bios": [
                "{level} 수준의 리더십 전문가입니다. 강한 팀을 만들 수 있도록 도와드리겠습니다.",
                "팀 관리 분야에서 {level} 멘토로 활동하고 있습니다. 노하우를 공유합니다."
            ],
            "mentee_bios": [
                "{interest}에 도전하는 {level} 멘티입니다. 좋은 리더가 되고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        },
        "취업 준비 및 이직": {
            "expertise": ["이직 전략", "네트워킹", "시장 분석"],
            "interests": ["이직 준비", "커리어 발전", "취업 정보"],
            "mentor_bios": [
                "{level} 수준의 이직 전문가입니다. 새로운 기회를 찾도록 도와드리겠습니다.",
                "네트워킹 분야에서 {level} 멘토로 활동하고 있습니다. 인맥을 넓혀봐요."
            ],
            "mentee_bios": [
                "{interest}를 고민하는 {level} 멘티입니다. 더 나은 직장을 찾고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 준비하고 있습니다."
            ]
        }
    },
    "IT/기술": {
        "웹 개발": {
            "expertise": ["프론트엔드 개발", "백엔드 개발", "풀스택 개발"],
            "interests": ["웹 프로그래밍", "프론트엔드 기술", "백엔드 구조"],
            "mentor_bios": [
                "{level} 수준의 웹 개발자입니다. 최신 웹 기술을 함께 나눠요.",
                "웹 개발 분야에서 {level} 멘토로 활동하고 있습니다. 지식을 공유하겠습니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 개발 능력을 키우고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        },
        "데이터 분석": {
            "expertise": ["데이터 전처리", "통계 분석", "머신러닝"],
            "interests": ["데이터 사이언스", "통계 학습", "AI 기술"],
            "mentor_bios": [
                "{level} 수준의 데이터 분석 전문가입니다. 데이터의 가치를 함께 발견해요.",
                "데이터 사이언스 분야에서 {level} 멘토로 활동하고 있습니다. 경험을 나누겠습니다."
            ],
            "mentee_bios": [
                "{interest}에 열정을 가진 {level} 멘티입니다. 분석 능력을 향상시키고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 공부하고 있습니다."
            ]
        },
        "사이버 보안": {
            "expertise": ["네트워크 보안", "시스템 보안", "정보 보안"],
            "interests": ["보안 기술", "해킹 방어", "정보 보호"],
            "mentor_bios": [
                "{level} 수준의 사이버 보안 전문가입니다. 안전한 시스템을 구축해요.",
                "정보 보안 분야에서 {level} 멘토로 활동하고 있습니다. 지식을 나누겠습니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 보안 전문가가 되고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        },
        "인공지능(AI) 연구": {
            "expertise": ["딥러닝", "자연어 처리", "컴퓨터 비전"],
            "interests": ["AI 학습", "딥러닝 기술", "인공지능 연구"],
            "mentor_bios": [
                "{level} 수준의 인공지능 전문가입니다. 미래 기술을 함께 연구해요.",
                "AI 분야에서 {level} 멘토로 활동하고 있습니다. 혁신을 만들어봐요."
            ],
            "mentee_bios": [
                "{interest}에 열정을 가진 {level} 멘티입니다. AI 전문가가 되고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 공부하고 있습니다."
            ]
        },
        "클라우드 컴퓨팅": {
            "expertise": ["AWS", "Azure", "구글 클라우드"],
            "interests": ["클라우드 서비스", "인프라 관리", "DevOps"],
            "mentor_bios": [
                "{level} 수준의 클라우드 컴퓨팅 전문가입니다. 클라우드 기술을 공유합니다.",
                "클라우드 분야에서 {level} 멘토로 활동하고 있습니다. 함께 배워봐요."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 클라우드 기술을 습득하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        }
    },
    "금융/경제": {
        "재테크/투자": {
            "expertise": ["주식 투자", "부동산 투자", "금융 상품"],
            "interests": ["투자 방법", "재테크 전략", "수익 창출"],
            "mentor_bios": [
                "{level} 수준의 투자 전문가입니다. 재테크 노하우를 공유합니다.",
                "금융 투자 분야에서 {level} 멘토로 활동하고 있습니다. 성공적인 투자를 도와드리겠습니다."
            ],
            "mentee_bios": [
                "{interest}에 열정을 가진 {level} 멘티입니다. 자산을 늘리고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        },
        "개인 금융 관리": {
            "expertise": ["재무 계획", "예산 관리", "저축 방법"],
            "interests": ["금융 계획", "예산 세우기", "재무 건전성"],
            "mentor_bios": [
                "{level} 수준의 개인 금융 전문가입니다. 건전한 재정을 만들어봐요.",
                "재무 관리 분야에서 {level} 멘토로 활동하고 있습니다. 팁을 공유합니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 재정을 관리하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        },
        "법률 상담": {
            "expertise": ["계약법", "노동법", "지적 재산권"],
            "interests": ["법률 지식", "권리 보호", "법적 분쟁 해결"],
            "mentor_bios": [
                "{level} 수준의 법률 전문가입니다. 법률 문제를 해결해드리겠습니다.",
                "법률 상담 분야에서 {level} 멘토로 활동하고 있습니다. 도움을 드립니다."
            ],
            "mentee_bios": [
                "{interest}를 필요로 하는 {level} 멘티입니다. 법률 지식을 얻고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 해결하고 있습니다."
            ]
        },
        "부동산 투자": {
            "expertise": ["부동산 시장 분석", "투자 전략", "부동산 관리"],
            "interests": ["부동산 정보", "투자 기회", "자산 증식"],
            "mentor_bios": [
                "{level} 수준의 부동산 투자 전문가입니다. 성공적인 투자를 도와드리겠습니다.",
                "부동산 분야에서 {level} 멘토로 활동하고 있습니다. 경험을 공유합니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 부동산 투자를 배우고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        },
        "세무/회계": {
            "expertise": ["세금 절약 전략", "회계 원리", "재무 보고"],
            "interests": ["세무 관리", "회계 지식", "재무 이해"],
            "mentor_bios": [
                "{level} 수준의 세무/회계 전문가입니다. 재무를 명확히 해드립니다.",
                "세무 분야에서 {level} 멘토로 활동하고 있습니다. 지식을 나눕니다."
            ],
            "mentee_bios": [
                "{interest}를 필요로 하는 {level} 멘티입니다. 재무 지식을 쌓고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 공부하고 있습니다."
            ]
        }
    },
    "자기계발/멘탈케어": {
        "인간관계": {
            "expertise": ["대인관계 기술", "소통 방법", "네트워킹"],
            "interests": ["대인 관계 개선", "소통 능력", "인맥 형성"],
            "mentor_bios": [
                "{level} 수준의 인간관계 전문가입니다. 사람과의 연결을 돕겠습니다.",
                "소통 분야에서 {level} 멘토로 활동하고 있습니다. 함께 성장해요."
            ],
            "mentee_bios": [
                "{interest}를 바라는 {level} 멘티입니다. 관계를 개선하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        },
        "시간 관리": {
            "expertise": ["효율적인 시간 사용", "우선순위 설정", "목표 달성"],
            "interests": ["시간 관리", "생산성 향상", "목표 계획"],
            "mentor_bios": [
                "{level} 수준의 시간 관리 전문가입니다. 하루를 알차게 만들어봐요.",
                "생산성 분야에서 {level} 멘토로 활동하고 있습니다. 팁을 공유합니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 시간을 잘 활용하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 개선하고 있습니다."
            ]
        },
        "정신 건강": {
            "expertise": ["스트레스 관리", "심리 상담", "마인드풀니스"],
            "interests": ["멘탈 케어", "스트레스 해소", "심리적 안정"],
            "mentor_bios": [
                "{level} 수준의 정신 건강 전문가입니다. 마음의 안정을 찾아드리겠습니다.",
                "심리 상담 분야에서 {level} 멘토로 활동하고 있습니다. 함께 힐링해요."
            ],
            "mentee_bios": [
                "{interest}를 원하고 있는 {level} 멘티입니다. 마음을 돌보고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        },
        "생활 건강": {
            "expertise": ["건강 관리", "운동 계획", "식습관 개선"],
            "interests": ["건강 유지", "운동 습관", "영양 관리"],
            "mentor_bios": [
                "{level} 수준의 생활 건강 전문가입니다. 건강한 삶을 만들어봐요.",
                "건강 관리 분야에서 {level} 멘토로 활동하고 있습니다. 조언을 드립니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 건강을 지키고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        },
        "커뮤니케이션": {
            "expertise": ["의사소통 기술", "프레젠테이션 스킬", "공감 능력"],
            "interests": ["소통 능력", "스피치 향상", "대화 기술"],
            "mentor_bios": [
                "{level} 수준의 커뮤니케이션 전문가입니다. 소통의 달인이 되어봐요.",
                "의사소통 분야에서 {level} 멘토로 활동하고 있습니다. 함께 배우겠습니다."
            ],
            "mentee_bios": [
                "{interest}를 향상시키고 싶은 {level} 멘티입니다. 잘 말하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        }
    },
    "예술/취미": {
        "미술": {
            "expertise": ["그림 그리기", "디지털 아트", "조소"],
            "interests": ["미술 활동", "그림 배우기", "창작"],
            "mentor_bios": [
                "{level} 수준의 미술가입니다. 예술의 세계로 안내합니다.",
                "미술 분야에서 {level} 멘토로 활동하고 있습니다. 창의성을 키워봐요."
            ],
            "mentee_bios": [
                "{interest}에 열정을 가진 {level} 멘티입니다. 예술을 즐기고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        },
        "독서": {
            "expertise": ["문학 감상", "비평", "독서 토론"],
            "interests": ["책 읽기", "문학 이해", "지식 습득"],
            "mentor_bios": [
                "{level} 수준의 독서가입니다. 책 속의 지혜를 나눕니다.",
                "독서 분야에서 {level} 멘토로 활동하고 있습니다. 함께 읽어요."
            ],
            "mentee_bios": [
                "{interest}를 사랑하는 {level} 멘티입니다. 더 많이 알고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 즐기고 있습니다."
            ]
        },
        "스포츠": {
            "expertise": ["축구", "농구", "테니스"],
            "interests": ["운동", "스포츠 배우기", "건강한 생활"],
            "mentor_bios": [
                "{level} 수준의 스포츠 코치입니다. 활기찬 삶을 만들어봐요.",
                "스포츠 분야에서 {level} 멘토로 활동하고 있습니다. 함께 운동해요."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 운동을 즐기고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        },
        "음악/작곡": {
            "expertise": ["악기 연주", "작곡", "음악 이론"],
            "interests": ["음악 감상", "악기 배우기", "작곡"],
            "mentor_bios": [
                "{level} 수준의 음악가입니다. 음악의 즐거움을 나눠요.",
                "음악 분야에서 {level} 멘토로 활동하고 있습니다. 함께 연주해요."
            ],
            "mentee_bios": [
                "{interest}에 열정을 가진 {level} 멘티입니다. 음악을 배우고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        },
        "사진/미디어": {
            "expertise": ["사진 촬영", "영상 편집", "미디어 제작"],
            "interests": ["사진 기술", "영상 제작", "미디어 활용"],
            "mentor_bios": [
                "{level} 수준의 미디어 전문가입니다. 멋진 작품을 만들어봐요.",
                "사진/미디어 분야에서 {level} 멘토로 활동하고 있습니다. 노하우를 공유합니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 작품을 만들고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        }
    },
    "기타": {
        "식단/영양": {
            "expertise": ["영양 관리", "식단 계획", "건강 요리"],
            "interests": ["건강한 식단", "요리 배우기", "영양 지식"],
            "mentor_bios": [
                "{level} 수준의 영양 전문가입니다. 건강한 식습관을 만들어봐요.",
                "식단 관리 분야에서 {level} 멘토로 활동하고 있습니다. 팁을 공유합니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 건강하게 먹고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        },
        "패션/스타일링": {
            "expertise": ["패션 코디", "스타일 제안", "이미지 메이킹"],
            "interests": ["패션 센스", "스타일 변화", "자신감 향상"],
            "mentor_bios": [
                "{level} 수준의 패션 전문가입니다. 멋진 스타일을 찾아봐요.",
                "스타일링 분야에서 {level} 멘토로 활동하고 있습니다. 조언을 드립니다."
            ],
            "mentee_bios": [
                "{interest}를 원하는 {level} 멘티입니다. 멋있게 보이고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        },
        "여행 계획": {
            "expertise": ["여행 코스 설계", "예산 계획", "문화 이해"],
            "interests": ["여행 준비", "새로운 경험", "세계 탐험"],
            "mentor_bios": [
                "{level} 수준의 여행 전문가입니다. 잊지 못할 여행을 만들어봐요.",
                "여행 계획 분야에서 {level} 멘토로 활동하고 있습니다. 함께 떠나요."
            ],
            "mentee_bios": [
                "{interest}를 꿈꾸는 {level} 멘티입니다. 여행을 가고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 준비하고 있습니다."
            ]
        },
        "자녀 교육/양육": {
            "expertise": ["교육 방법", "양육 기술", "발달 심리"],
            "interests": ["육아 방법", "자녀 교육", "가족 관계"],
            "mentor_bios": [
                "{level} 수준의 양육 전문가입니다. 행복한 가정을 만들어봐요.",
                "자녀 교육 분야에서 {level} 멘토로 활동하고 있습니다. 도움을 드립니다."
            ],
            "mentee_bios": [
                "{interest}에 관심 있는 {level} 멘티입니다. 좋은 부모가 되고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 배우고 있습니다."
            ]
        },
        "연애/결혼 상담": {
            "expertise": ["관계 구축", "소통 방법", "문제 해결"],
            "interests": ["연애 조언", "결혼 준비", "관계 개선"],
            "mentor_bios": [
                "{level} 수준의 연애/결혼 상담 전문가입니다. 행복한 관계를 도와드리겠습니다.",
                "관계 상담 분야에서 {level} 멘토로 활동하고 있습니다. 조언을 드립니다."
            ],
            "mentee_bios": [
                "{interest}를 바라는 {level} 멘티입니다. 행복한 연애를 하고 싶어요.",
                "안녕하세요, {level} 단계의 {interest}를 시작하고 있습니다."
            ]
        }
    }
}



In [2]:
synonyms = {
    # 학업/교육
    "진로상담": ["진로 상담", "커리어 상담", "직업 상담", "진로 지도", "진로 코칭", "커리어 코칭", "커리어 컨설팅"],
    "취업준비": ["취업 준비", "구직 활동", "취업 전략", "구직 준비", "취업 코칭", "취업 지원", "구인 구직"],
    "면접대비": ["면접 대비", "면접 준비", "인터뷰 준비", "면접 연습", "면접 스킬", "인터뷰 스킬", "면접 기법"],
    "대학입시및유학": ["대학 입시", "유학 준비", "시험 대비", "입시 준비", "유학 정보", "해외 유학", "입시 컨설팅"],
    "자격증": ["자격증 취득", "자격증 공부", "시험 준비", "자격증 시험", "자격증 준비", "자격 시험", "자격증 합격"],

    # 경력/직업
    "경력전환": ["경력 전환", "커리어 전환", "직무 변경", "직업 변경", "커리어 체인지", "직무 전환", "이직"],
    "직장인실무": ["직장인 실무", "업무 스킬", "직무 능력", "실무 능력", "업무 능력", "직장 스킬", "직무 스킬"],
    "직장내문제해결": ["직장 내 문제 해결", "직장 문제 해결", "갈등 관리", "직장 고민 해결", "직장 스트레스 관리", "직장 갈등 해결"],
    "리더십및팀관리": ["리더십", "팀 관리", "팀 빌딩", "리더십 개발", "팀워크 향상", "조직 관리", "관리 능력"],
    "취업준비및이직": ["취업 준비", "이직 준비", "구직 활동", "커리어 발전", "직업 전환", "이직 전략", "취업 지원"],

    # IT/기술
    "웹개발": ["웹 개발", "프론트엔드", "백엔드", "풀스택", "웹 프로그래밍", "웹 디자인", "웹 코딩"],
    "데이터분석": ["데이터 분석", "데이터 사이언스", "통계 분석", "데이터 처리", "빅데이터", "데이터 마이닝"],
    "사이버보안": ["사이버 보안", "정보 보안", "네트워크 보안", "시스템 보안", "보안 전문가", "해킹 방지"],
    "인공지능ai연구": ["인공지능", "AI", "딥러닝", "머신러닝", "자연어 처리", "컴퓨터 비전"],
    "클라우드컴퓨팅": ["클라우드 컴퓨팅", "AWS", "Azure", "구글 클라우드", "클라우드 서비스", "클라우드 인프라"],

    # 금융/경제
    "재테크투자": ["재테크", "투자", "주식 투자", "부동산 투자", "금융 투자", "자산 관리", "재무 설계"],
    "개인금융관리": ["개인 금융 관리", "재무 계획", "예산 관리", "자산 관리", "재정 관리", "가계부 작성"],
    "법률상담": ["법률 상담", "법률 자문", "법률 지식", "법적 조언", "법률 지원", "법률 서비스"],
    "부동산투자": ["부동산 투자", "부동산 매매", "부동산 관리", "부동산 재테크", "부동산 개발", "부동산 임대"],
    "세무회계": ["세무", "회계", "재무 보고", "세금 관리", "회계 처리", "세금 절약", "재무 회계"],

    # 자기계발/멘탈케어
    "인간관계": ["인간 관계", "대인 관계", "관계 개선", "소통 능력", "인맥 형성", "관계 형성", "네트워킹"],
    "시간관리": ["시간 관리", "타임 매니지먼트", "우선순위 설정", "효율적인 시간 사용", "시간 활용", "시간 절약"],
    "정신건강": ["정신 건강", "멘탈 케어", "심리 상담", "스트레스 관리", "마음 건강", "정신 안정"],
    "생활건강": ["생활 건강", "건강 관리", "운동 습관", "식습관 개선", "웰빙", "건강한 삶"],
    "커뮤니케이션": ["커뮤니케이션", "의사소통", "프레젠테이션 스킬", "대화 기술", "말하기 능력", "소통 방법"],

    # 예술/취미
    "미술": ["미술", "그림 그리기", "디지털 아트", "조소", "회화", "예술 활동", "드로잉"],
    "독서": ["독서", "책 읽기", "문학 감상", "독서 토론", "독후감", "책 리뷰"],
    "스포츠": ["스포츠", "운동", "체육 활동", "피트니스", "스포츠 배우기", "건강한 생활"],
    "음악작곡": ["음악", "작곡", "악기 연주", "음악 이론", "노래 부르기", "뮤직 프로덕션"],
    "사진미디어": ["사진", "영상", "미디어", "영상 편집", "사진 촬영", "미디어 제작"],

    # 기타
    "식단영양": ["식단", "영양", "다이어트", "건강 식품", "식습관", "영양 관리", "식단 계획"],
    "패션스타일링": ["패션", "스타일링", "코디네이션", "의상", "이미지 메이킹", "패션 코디", "패션 센스"],
    "여행계획": ["여행 계획", "여행 준비", "여행 코스", "여행 일정", "여행 정보", "여행 가이드"],
    "자녀교육양육": ["자녀 교육", "양육", "육아", "발달 심리", "아이 교육", "부모 교육"],
    "연애결혼상담": ["연애 상담", "결혼 상담", "관계 상담", "연애 조언", "연애 문제", "결혼 준비"],

    # 추가 상세 동의어
    "프론트엔드": ["프론트엔드", "HTML", "CSS", "JavaScript", "React", "Vue.js", "Angular"],
    "백엔드": ["백엔드", "서버 개발", "데이터베이스", "API 개발", "Node.js", "Django", "Spring"],
    "풀스택": ["풀스택", "프론트엔드 및 백엔드", "전체 웹 개발", "Full Stack"],
    "데이터사이언스": ["데이터 사이언스", "데이터 분석", "빅데이터", "데이터 마이닝", "통계 분석"],
    "머신러닝": ["머신러닝", "기계 학습", "딥러닝", "인공 신경망", "ML", "Machine Learning"],
    "네트워크보안": ["네트워크 보안", "사이버 보안", "해킹 방지", "정보 보호", "보안 관리"],
    "딥러닝": ["딥러닝", "심층 학습", "인공 신경망", "Deep Learning"],
    "자연어처리": ["자연어 처리", "NLP", "텍스트 마이닝", "언어 처리", "언어 인식"],
    "컴퓨터비전": ["컴퓨터 비전", "영상 처리", "이미지 인식", "패턴 인식", "Computer Vision"],
    "클라우드서비스": ["클라우드 서비스", "클라우드 컴퓨팅", "Cloud Service", "Cloud Computing"],
    "재무계획": ["재무 계획", "재무 설계", "자산 관리", "재정 계획", "재무 전략"],
    "예산관리": ["예산 관리", "예산 계획", "지출 관리", "가계부 작성", "재무 관리"],
    "저축방법": ["저축 방법", "저축 전략", "저축 습관", "저축 노하우", "재산 증식"],
    "스트레스관리": ["스트레스 관리", "스트레스 해소", "마음 다스리기", "긴장 완화", "스트레스 해결"],
    "마인드풀니스": ["마인드풀니스", "명상", "요가", "심신 안정", "마음챙김"],
    "운동계획": ["운동 계획", "운동 프로그램", "피트니스", "트레이닝", "운동 루틴"],
    "식습관개선": ["식습관 개선", "다이어트", "건강 식단", "영양 관리", "식단 조절"],
    "그림그리기": ["그림 그리기", "스케치", "수채화", "유화", "드로잉"],
    "디지털아트": ["디지털 아트", "포토샵", "일러스트레이터", "CG", "컴퓨터 그래픽"],
    "조소": ["조소", "조각", "모형 제작", "클레이 아트", "입체 예술"],
    "악기연주": ["악기 연주", "피아노", "기타", "바이올린", "드럼", "플루트"],
    "음악이론": ["음악 이론", "화성학", "음악 작곡", "음악 기초", "음악 분석"],
    "사진촬영": ["사진 촬영", "카메라 기술", "촬영 기법", "사진 예술", "포토그래피"],
    "영상편집": ["영상 편집", "프리미어 프로", "애프터 이펙트", "영상 제작", "동영상 편집"],
    "영양관리": ["영양 관리", "영양 섭취", "영양소", "식단 관리", "다이어트"],
    "건강요리": ["건강 요리", "웰빙 요리", "다이어트 요리", "저칼로리 요리", "영양 식단"],
    "패션코디": ["패션 코디", "스타일링", "의상 조합", "패션 트렌드", "스타일 코디"],
    "이미지메이킹": ["이미지 메이킹", "퍼스널 브랜딩", "외모 개선", "스타일 개선", "이미지 개선"],
    "여행코스설계": ["여행 코스 설계", "여행 계획", "여행 일정 짜기", "여행 일정", "여행 준비"],
    "육아방법": ["육아 방법", "아이 양육", "자녀 교육", "부모 교육", "아이 돌보기"],
    "발달심리": ["발달 심리", "아동 발달", "심리 발달", "심리 성장", "발달 단계"],
    "관계구축": ["관계 구축", "인간 관계", "관계 형성", "네트워킹", "친목 도모"],
    "문제해결": ["문제 해결", "갈등 해결", "분쟁 조정", "해결책 찾기", "문제 분석"],
    "해외유학": ["해외 유학", "유학 준비", "유학 정보", "유학 상담", "유학 컨설팅"],
    "취업코칭": ["취업 코칭", "취업 지도", "취업 컨설팅", "취업 지원"],
    "이력서작성": ["이력서 작성", "자소서 작성", "이력서 개선", "이력서 컨설팅"],
    "프레젠테이션스킬": ["프레젠테이션 스킬", "발표 기술", "스피치 능력", "프레젠테이션 능력"],
    "커리어코칭": ["커리어 코칭", "진로 코칭", "커리어 컨설팅", "커리어 개발"],
    "조직문화이해": ["조직 문화 이해", "기업 문화 이해", "회사 문화 이해", "조직 이해"],
    "성과관리": ["성과 관리", "업적 관리", "목표 관리", "성과 향상"],
    "시장분석": ["시장 분석", "업계 분석", "트렌드 분석", "시장 조사"],
    "보안기술": ["보안 기술", "보안 시스템", "보안 솔루션", "보안 프로그램"],
    "AI학습": ["AI 학습", "인공지능 학습", "AI 공부", "인공지능 연구"],
    "DevOps": ["DevOps", "개발 운영", "CI/CD", "자동화 배포"],
}


In [3]:
# 한국어 불용어 사전
korean_stopwords = [
    '의', '가', '이', '은', '들', '는', '좀', '잘', '걍',
    '과', '도', '를', '으로', '자', '에', '와', '한', '하다',
    '및', '수', '등', '것', '게', '만', '듯', '지', '때',
    '내', '너', '우리', '저', '더', '그러나', '하지만', '그리고',
    '또', '그래서', '그런데', '저는', '제가', '하지만', '이런', '저런',
    # 필요에 따라 추가
]


In [4]:
import json
import random
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt

# 멘토와 멘티 이름 리스트 

# 레벨 목록 

# 카테고리와 중분류 정의 

# 시놉시스(동의어) 사전 




In [5]:
# 키워드 확장 함수
def expand_keywords(keywords, synonyms_dict):
    expanded = set()
    for keyword in keywords:
        keyword_key = keyword.replace(" ", "").lower()
        if keyword_key in synonyms_dict:
            expanded.update(synonyms_dict[keyword_key])
        else:
            expanded.add(keyword)
    return expanded

# 키워드 전처리 함수
def preprocess_keywords(keywords):
    processed = set()
    for keyword in keywords:
        keyword_clean = re.sub(r'\s+', '', keyword).lower()
        processed.add(keyword_clean)
    return processed

# 멘토 프로필 생성 함수
def generate_mentor(category_name, subcategory_name):
    subcategory = categories[category_name][subcategory_name]
    name = random.choice(mentor_names)
    expertise_list = subcategory["expertise"]
    expertise_sample = random.sample(expertise_list, k=random.randint(1, len(expertise_list)))
    level = random.choice([2, 3])  # 중급 또는 고급
    bio_template = random.choice(subcategory["mentor_bios"])
    bio = bio_template.format(
        expertise=", ".join(expertise_sample),
        level=levels[level]
    )
    return {
        "name": name,
        "expertise": expertise_sample,
        "level": level,
        "bio": bio
    }

# 멘티 프로필 생성 함수
def generate_mentee(category_name, subcategory_name):
    subcategory = categories[category_name][subcategory_name]
    name = random.choice(mentee_names)
    interests_list = subcategory["interests"]
    interests_sample = random.sample(interests_list, k=random.randint(1, len(interests_list)))
    level = random.choice([1, 2])  # 초급 또는 중급
    bio_template = random.choice(subcategory["mentee_bios"])
    bio = bio_template.format(
        interest=", ".join(interests_sample),
        level=levels[level]
    )
    return {
        "name": name,
        "interests": interests_sample,
        "level": level,
        "bio": bio
    }


In [6]:
# Okt 형태소 분석기 객체 생성
okt = Okt()

# 명사 추출 함수 (불용어 제거 포함)
def extract_nouns(text):
    nouns = okt.nouns(text)
    # 불용어 제거
    filtered_nouns = [noun for noun in nouns if noun not in korean_stopwords and len(noun) > 1]
    return ' '.join(filtered_nouns)


In [7]:
# 모든 텍스트 수집
all_texts = []

for category in categories.values():
    for subcategory in category.values():
        all_texts.extend(subcategory["mentor_bios"])
        all_texts.extend(subcategory["mentee_bios"])
        all_texts.extend(subcategory["expertise"])
        all_texts.extend(subcategory["interests"])

# 명사 추출하여 전체 텍스트 생성
all_nouns = [extract_nouns(text) for text in all_texts]

# 벡터라이저 학습 (불용어 제거를 위해 stop_words 인자로 전달)
vectorizer = TfidfVectorizer(stop_words=korean_stopwords)
vectorizer.fit(all_nouns)


TfidfVectorizer(stop_words=['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과',
                            '도', '를', '으로', '자', '에', '와', '한', '하다', '및', '수',
                            '등', '것', '게', '만', '듯', '지', '때', '내', '너', '우리', ...])

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(text1, text2):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])
    return similarity[0][0]


In [9]:
def determine_match(mentor, mentee):
    # 멘토와 멘티의 텍스트 결합 (소개글 + 전문 분야/관심사)
    mentor_text = mentor["bio"] + " " + " ".join(mentor.get("expertise", []))
    mentee_text = mentee["bio"] + " " + " ".join(mentee.get("interests", []))
    
    # 명사 추출
    mentor_nouns = extract_nouns(mentor_text)
    mentee_nouns = extract_nouns(mentee_text)
    
    # TF-IDF 벡터화 (벡터라이저 재사용)
    mentor_tfidf = vectorizer.transform([mentor_nouns])
    mentee_tfidf = vectorizer.transform([mentee_nouns])
    
    # 코사인 유사도 계산
    similarity = cosine_similarity(mentor_tfidf, mentee_tfidf)
    bio_similarity = similarity[0][0]
    
    # 레벨 차이에 따른 점수 계산 (가중치 낮게 설정)
    level_difference = abs(mentor["level"] - mentee["level"])
    if level_difference == 0:
        level_score = 0.2
    elif level_difference == 1:
        level_score = 0.1
    else:
        level_score = 0  # 레벨 차이가 2 이상이면 0점
    
    # 총 매칭 점수 계산
    total_score = bio_similarity * 0.9 + level_score  # bio_similarity 가중치를 높게 설정
    
    # 매칭 기준 설정 (총 점수가 0.5 이상이면 매칭)
    if total_score >= 0.5:
        return 1  # 매칭 성공
    else:
        return 0  # 매칭 실패


In [10]:
# 데이터 생성 함수
def generate_data(num_samples_per_subcategory):
    data_list = []
    for category_name, subcategories in categories.items():
        for subcategory_name in subcategories.keys():
            for _ in range(num_samples_per_subcategory):
                mentor = generate_mentor(category_name, subcategory_name)
                mentee = generate_mentee(category_name, subcategory_name)
                match = determine_match(mentor, mentee)
                data_pair = {
                    "category": category_name,
                    "subcategory": subcategory_name,
                    "mentor": mentor,
                    "mentee": mentee,
                    "match": match
                }
                data_list.append(data_pair)
    return data_list

In [11]:
# 데이터 생성 및 저장
num_samples_per_subcategory = 1000  # 각 중분류별로 생성할 데이터 수
data_list = generate_data(num_samples_per_subcategory)

# JSON 파일로 저장
with open("mentor_mentee_data.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)

# 매칭된 쌍의 수 확인
matched_pairs = [pair for pair in data_list if pair['match'] == 1]
print(f"매칭된 쌍의 수: {len(matched_pairs)}")


매칭된 쌍의 수: 8106
